과제: 파이썬 머신러닝 완벽가이드 ch9.5 필사

마감: 5월 13일 월요일 18:30

579p~ 590p 필사하여 링크를 올려주세요.

​

​

p.589

수정전)

movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

수정후)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

---------
# Chapter9. 추천 시스템

## 9-5. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
콘텐츠 기반 필터링 : 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성, 구성요소 등을 가진 다른 영화를 추천하는 것. 즉 영화 간의 유사성을 판단하는 기준이 영화를 구성하는 콘텐츠(장르, 감독, 배우, 평점, 키워드 등)를 기반으로 하는 것.

이번에는 영화 장르 속성을 기반으로 만들어 볼 것이다. 장르 칼럼 값의 유사도를 비교한 뒤 그 중 높은 평점을 가지는 영화를 추천하는 방식.

-----
### 데이터 로딩 및 가공

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('./data/tmdb_5000/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출하여 새로운 DF로 만든다.
- id
- title : 영화 제목
- genres : 영화 장르
- vote_average : 평균 평점
- vote_count : 평점 투표 수
- popularity : 영화 인기
- keywords : 주요 키워드
- overview : 영화에 대한 개요 설명

In [3]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

주의해야 할 칼럼 : genres, keywords

genres, keywords 등의 칼럼에는 파이썬 list 내부에 여러 개의 딕셔너리가 있는 형태로 구성되어 있다. 이는 한 번에 여러 개의 값을 표현하기 위한 표기 방식이다. 따라서 이런 칼럼을 가공해야 필요한 정보를 추출할 수가 있다. 이러한 칼럼들을 살펴보자.

In [4]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출해보자.

파이썬 ast 모듈의 literal_eval() 이라는 함수를 이용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다. Series 객체의 apply()를 이용해서 함수를 적용해보자.

In [5]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

이제 genres는 문자열이 아니라 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다.

이제는 genres 칼럼에서 'Action', 'Adventure'와 같은 실제 장르 명만 리스트 객체로 추출해보자.

In [6]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


-----
### 장르 콘텐츠 유사도 측정

genres 칼럼은 여러 개의 개별 장르가 리스트로 구성되어 있다.

장르별 유사도를 측정하는 가장 간단한 방법은 genres를 문자열로 변경한 뒤 이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것이다. genres 칼럼을 기반으로 하는 콘텐츠 기반으로 하는 콘텐츠 기반 필터링은 다음과 같은 단계로 구현한다

1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환한다.
2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교한다. 이를 위해 데이터 세트의 레코드별로 타 레코드와 genres에서 코사인 유사도 값을 가지는 객체를 생성한다.
3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다.

-----

1. genres 칼럼을 문자열 변환한 뒤 CountVectorizer를 이용해 피처 벡터 행렬로 만든다.
-> apply(lambda x:(' ').join(x)) 적용하여 개별 요소를 공백 문자로 구분하는 문자열로 변환해 genres_literal이라는 별도의 칼럼으로 저장한다.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 것을 문자열로 변환한다
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df = 0.0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


4803개의 레코드와 276개의 개별 단어 피처로 구성된 피처 벡터 행렬이 만들어졌다. 이렇게 생성된 피처 벡터 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도를 계산한다.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


genre_sim 객체는 movies_df의 행별 장르 유사도 값을 갖는 것이다. 이 genre_sim을 이용하여 장르 유사도가 높은 순으로 다른 레코드를 추출한다.

-> genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출한다 (argsort 이용)

In [11]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


이는 0번 레코드의 경우, 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 높은 장르 유사도를 갖고, 그 다음으로 813번이 높으며, 가장 유사도가 낮은 레코드는 2401번 레코드임을 의미한다.

-----
### 장르 콘텐츠 필터링을 이용한 영화 추천

이제 장르 유사도에 따라 영화를 추천하는 함수인 find_sim_movie()를 만들어보자.

<함수 입력 인자>
- df : 기반 데이터인 movies_df
- sorted_ind : 레코드별 장르 코사인 유사도 인덱스를 갖는 genre_sim_sorted_ind
- title_name : 고객이 선정한 추천 기준이 되는 영화 제목
- top_n : 추천할 영화 건수

In [16]:
# 인자로 입력된 movies_df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]

    # title_named를 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    # 추출된 top_n index 출력, top_n index는 2차원 데이터임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

이 함수를 이용해 영화 '대부'와 장르별로 유사한 영화 10개를 추천해보자.

In [17]:
similar_movie = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movie[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


---------
좀 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 추천하는 방식으로 변경해보자.
영화의 평점 정보인 'vote_average'를 사용하는데, 이때 1명, 2명 등 소수의 관객이 특정 영화에 평점을 부여할 경우 데이터가 왜곡될 수 있다. 이를 확인하기 위해 vote_count를 살펴보자.

In [18]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


쇼생크 탈출이나 대부 같은 명작보다 높은 순위에 듣보잡 영화가 더 높은 평점으로 있다. 모두 평가 횟수가 적다. 이처럼 왜곡된 데이터를 회피하기 위해 평점에 평가 횟수를 반영하는 새로운 평가 방식을 도입하자.

유명한 영화 평점 사이트인 IMDB에서 사용하는 Weighted Rating 방식을 이용하여 평점을 새롭게 부여하자.

<center>Weighted Rating = (v/(v+me)) * R + (m/v+m) * C</center>  

- v : 개별 영화에 평점을 투표한 횟수
- m : 평점을 부여하기 위한 최소 투표 횟수
- R : 개별 영화에 대한 평균 평점
- C : 전체 영화에 대한 평균 평점

-----
- V : movies_df['vote_count]
- R : movies_df['vote_average]
- C : movies_df['vote_average'].mean()
- m : 투표 횟수에 따른 가중치 조절 (높을수록 투표횟수가 많은 영화에 가중 평점 부여)
-> 상위 60%에 해당하는 횟수를 기준으로 설정하자.


In [19]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C : ', round(C, 3), 'm : ', round(m, 3))

C :  6.092 m :  370.2


기존 평점을 새로운 가중 평점으로 변경하는 함수 weighted_vote_average()를 생성하자.

In [24]:
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ((v/(v+m))*R + ((m/(m+v))*C))

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis = 1)

새롭게 부여된 weighted_vote가 높은 순으로 상위 10개의 영화를 추출해보자

In [25]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


-------
이렇게 새롭게 정의된 평점 기준에 따라 함수를 변경하고 다시 영화를 추천해보자.

In [29]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
